In [1]:
%matplotlib inline

# PYTORCH: DEFINING NEW AUTOGRAD FUNCTIONS

同样的回归问题，但这次我们要实现一个自己的ReLU函数，并增加它的自动求导功能。

In [ ]:
import torch 

class MyReLU(torch.autograd.Function):
    """
    通过继承torch.autograd.Function来实现我们的ReLU函数。
    需要实现它的正向和反向传播。
    """
    
    @staticmethod
    def forward(ctx, input):
        """
        在forward函数中，我们获取一个Tensor作为参数input，
        """